In [122]:
# 필요한 라이브러리 불러오기
import pandas as pd

# 훈련 데이터 파일 경로
train_data_path = r'C:\Users\82106\Desktop\데이터분석 프로젝트\신약개발 AI경진대회\data\open\train.csv'

# 훈련 데이터 불러오기
train_data = pd.read_csv(train_data_path)


## 결측치 처리

In [123]:
# 결측치 확인
train_data.isnull().sum()


id                            0
SMILES                        0
MLM                           0
HLM                           0
AlogP                         2
Molecular_Weight              0
Num_H_Acceptors               0
Num_H_Donors                  0
Num_RotatableBonds            0
LogD                          0
Molecular_PolarSurfaceArea    0
dtype: int64

In [124]:
# 결측치를 채우는 예시: 평균 값으로 채우기
train_data['AlogP'].fillna(train_data['AlogP'].mean(), inplace=True)

In [125]:
train_data.isnull().sum()

id                            0
SMILES                        0
MLM                           0
HLM                           0
AlogP                         0
Molecular_Weight              0
Num_H_Acceptors               0
Num_H_Donors                  0
Num_RotatableBonds            0
LogD                          0
Molecular_PolarSurfaceArea    0
dtype: int64

## SMILES 변수 RDKit 를 이용하여 분자 지문(Molecular Fingerprints)을 생성

In [72]:
pip install rdkit-pypi


Note: you may need to restart the kernel to use updated packages.


In [96]:
from rdkit import Chem
from rdkit.Chem import AllChem

# SMILES 데이터를 RDKit 분자 객체로 변환
molecules = [Chem.MolFromSmiles(smiles) for smiles in train_data['SMILES']]

# 분자 지문(fingerprint) 생성
fingerprints = [AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024) for mol in molecules]

# 분자 지문을 배열로 변환
fingerprint_array = [list(fp.ToBitString()) for fp in fingerprints]

# 배열을 DataFrame으로 변환
fingerprint_df = pd.DataFrame(fingerprint_array, columns=[f'bit_{i}' for i in range(1024)])

# 원래 데이터프레임과 분자 지문 데이터프레임을 병합
merged_df = pd.concat([train_data, fingerprint_df], axis=1)


## 스케일링

In [126]:
from sklearn.preprocessing import StandardScaler

# 숫자형 특성 목록
numeric_features = ['AlogP', 'Molecular_Weight', 'Num_H_Acceptors', 'Num_H_Donors', 'Num_RotatableBonds', 'LogD', 'Molecular_PolarSurfaceArea']

# 특성 스케일링 (표준화)
scaler = StandardScaler()
train_data[numeric_features] = scaler.fit_transform(train_data[numeric_features])


## 불필요한 특성 삭제

In [127]:
from sklearn.model_selection import train_test_split

# 특성 (X)과 타겟 (y) 분리
X = train_data.drop(columns=['MLM'])
y = train_data['MLM']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [128]:
# 불필요한 특성 삭제 
X_train = X_train.drop(columns=['id'])
X_test = X_test.drop(columns=['id'])


# 모델 학습 LinearRegeression

In [129]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# 분자 지문을 포함한 DataFrame에서 타겟 변수와 특성을 선택
X = train_data.drop(columns=['MLM','SMILES'])  # 'MLM'과 'SMILES' 열을 제외한 특성 선택
y = train_data['MLM']  # 'MLM' 열을 타겟 변수로 선택

# 훈련 데이터와 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 불필요한 특성 삭제 
X_train = X_train.drop(columns=['id'])
X_test = X_test.drop(columns=['id'])

# 선형 회귀 모델 생성
model = LinearRegression()

# 모델 학습
model.fit(X_train, y_train)

# 테스트 데이터로 예측
y_pred = model.predict(X_test)

# 평균 제곱 오차(MSE) 계산
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')


Mean Squared Error: 662.2813003872544


# GradientBoosting

In [130]:
from sklearn.ensemble import GradientBoostingRegressor  # 회귀 모델을 사용할 경우
# 또는
# from sklearn.ensemble import GradientBoostingClassifier  # 분류 모델을 사용할 경우

# 그라디언트 부스팅 모델 생성
model = GradientBoostingRegressor()  # 또는 GradientBoostingClassifier()를 사용하여 분류 모델 생성
# 모델 학습
model.fit(X_train, y_train)
# 예측 수행
y_pred = model.predict(X_test)
from sklearn.metrics import mean_squared_error

# 회귀 모델의 예측 성능 평가 (예시: 평균 제곱 오차 MSE)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')


Mean Squared Error: 638.256593664714


# 나이브 베이즈 (Naive Bayes)

In [27]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# 이진 분류를 위한 임계값 설정 (예시: 0.5)
threshold = 0.5

# 연속형 타겟 변수를 이진 분류로 변환
y_binary = (y > threshold).astype(int)

# 데이터 전처리 및 훈련/테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.2, random_state=42)

# 가우시안 나이브 베이즈 모델 생성
model = GaussianNB()

# 모델 학습
model.fit(X_train, y_train)

# 예측 수행
y_pred = model.predict(X_test)

# 정확도 평가
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# 분류 보고서 출력
report = classification_report(y_test, y_pred)
print(report)


Accuracy: 0.14142857142857143
              precision    recall  f1-score   support

           0       0.14      1.00      0.24        97
           1       1.00      0.00      0.01       603

    accuracy                           0.14       700
   macro avg       0.57      0.50      0.13       700
weighted avg       0.88      0.14      0.04       700



# 커널 서포트 벡터 머신 (Kernel Support Vector Machine)

In [132]:
import numpy as np

# 임계값 설정
threshold = 0.5  # 예시 임계값

# 연속형 타겟 변수를 이진 분류로 변환
y_binary = np.where(y > threshold, 1, 0)

# 이제 y_binary를 이진 분류로 사용할 수 있습니다.


In [133]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVR

# SVR 모델 생성
model = SVR(kernel='rbf')  # RBF 커널 사용 (회귀 모델)

# SVR 모델 학습
model.fit(X_train, y_train)

# 예측 수행
y_pred = model.predict(X_test)

# 회귀 모델에서는 평균 제곱 오차 (MSE) 등을 사용하여 성능 평가
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')


Mean Squared Error: 696.8156905138584


In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

# SVR 모델 생성
model = SVR()

# 튜닝할 하이퍼파라미터 범위 설정 (더욱 제한된 범위)
param_grid = {
    'C': [1],  # C 값
    'kernel': ['rbf'],  # 커널 종류 (RBF만 사용)
    'gamma': [0.1],  # gamma 값
    'epsilon': [0.1]  # epsilon 값
}

# GridSearchCV를 사용하여 모델 튜닝
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터 확인
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# 최적의 모델을 얻기
best_model = grid_search.best_estimator_

# 테스트 데이터로 예측
y_pred = best_model.predict(X_test)

# 평균 제곱 오차 (MSE) 계산
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
